In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.float_format', '{:,.4f}'.format)

## Ejercicio 1

In [2]:
def VaR_APL(data: pd.DataFrame, posiciones: np.array, conf: float, long: bool):
    """ 
    Función que calcula el VaR y C-VaR de un portafolio de activos financieros ajustado por costos de liquidez.

    Args:
    data (pd.DataFrame): DataFrame con los bid y ask de los activos financieros.
    posiciones (np.array): Vector con las posiciones de los activos financieros.
    conf (float): Nivel de confianza.
    long (bool): True si el portafolio es largo, False si es corto.

    Returns:
    pd.DataFrame: DataFrame con el VaR y C-VaR ajustado por costos de liquidez.
    """

    data = data.sort_index()

    # Bid y Ask
    bid_columns = [col for col in data.columns if 'Bid' in col] # Selecciona las columnas que contienen 'Bid'
    ask_columns = [col for col in data.columns if 'Ask' in col] # Selecciona las columnas que contienen 'Ask'

    # Mid
    mid_columns = [f'Mid.{i}' for i in range(len(bid_columns))] # Se crea una lista con los nombres de las columnas de Mid
    data[mid_columns] = (data[bid_columns].values + data[ask_columns].values) / 2

    # Spreads
    spread_columns = [f'Spread.{i}' for i in range(len(bid_columns))] # Se crea una lista con los nombres de las columnas de Spread
    data[spread_columns] = (data[ask_columns].values - data[bid_columns].values) / data[mid_columns].values

    # Returns
    return_columns = [f'Return.{i}' for i in range(len(mid_columns))] # Se crea una lista con los nombres de las columnas de Return
    data[return_columns] = data[mid_columns].pct_change()

    # Weights
    value = posiciones * data[mid_columns].iloc[-1].values
    pv = np.sum(value)
    w = value / pv

    # Portfolio return
    data['port_ret'] = np.dot(data[return_columns], w)

    # VaR calculation
    var_pct = np.percentile(data['port_ret'].dropna(), 100 - conf*100) if long else np.percentile(data['port_ret'].dropna(), conf*100)
    var_cash = pv * var_pct

    # C-VaR calculation
    cvar_pct = data['port_ret'][data['port_ret'] < var_pct].dropna().mean() if long else data['port_ret'][data['port_ret'] > var_pct].dropna().mean()
    cvar_cash = pv * cvar_pct

    # Liquidity cost
    cl_prom = data[spread_columns].mean()
    cl_estr = np.percentile(data[spread_columns], 99, axis=0)

    # VaR adjusted by liquidity cost

    var_apl_prom, var_apl_estr = np.abs(((var_pct - np.dot(w, cl_prom), var_pct - np.dot(w, cl_estr)) if long 
                                else (var_pct + np.dot(w, cl_prom), var_pct + np.dot(w, cl_estr))))

    var_apl_prom_cash, var_apl_estr_cash = np.abs(((var_cash - np.dot(value, cl_prom), var_cash - np.dot(value, cl_estr)) if long 
                                            else (var_cash + np.dot(value, cl_prom), var_cash + np.dot(value, cl_estr))))
    
    # C-VaR adjusted by liquidity cost

    cvar_apl_prom, cvar_apl_estr = np.abs(((cvar_pct - np.dot(w, cl_prom), cvar_pct - np.dot(w, cl_estr)) if long
                                    else (cvar_pct + np.dot(w, cl_prom), cvar_pct + np.dot(w, cl_estr))))
    
    cvar_apl_prom_cash, cvar_apl_estr_cash = np.abs(((cvar_cash - np.dot(value, cl_prom), cvar_cash - np.dot(value, cl_estr)) if long
                                            else (cvar_cash + np.dot(value, cl_prom), cvar_cash + np.dot(value, cl_estr))))

    resultados = pd.DataFrame({
        'Métrica': ['VaR', 'VaR Ajustado Promedio', 'VaR Ajustado Estresado', 'C-VaR', 'C-VaR Ajustado Promedio', 'C-VaR Ajustado Estresado'],
        'Porcentaje': [np.abs(var_pct), var_apl_prom, var_apl_estr, np.abs(cvar_pct), cvar_apl_prom, cvar_apl_estr],
        'Cash': [np.abs(var_cash), var_apl_prom_cash, var_apl_estr_cash, np.abs(cvar_cash), cvar_apl_prom_cash, cvar_apl_estr_cash]
    })

    return resultados

## Ejercicio 2

### Ejercicio 2a

Asume que tienes un portafolio de FX con las siguientes posiciones:

- CORTO en el dólar 3.2 MDD (USD/MXN)
- CORTO en el yen japones 10 MDJ (JPY/MXN)
- CORTO en el euro 5.3 MDE (EUR/MXN)

Calcular las métricas de riesgo de mercado y liquidez del portafolio a un día con un 99% de confianza.

In [3]:
data1 = pd.read_excel('DataTarea5.xlsx', sheet_name=0)
data1 = data1.set_index('Date')
posiciones1 = np.array([3200000, 10000000, 5300000])
conf = 0.99
long = False

In [4]:
VaR_APL(data1, posiciones1, conf, long)

,Métrica,Porcentaje,Cash
0,VaR,0.0261,"4,656,957.0170"
1,VaR Ajustado Promedio,0.0266,"4,734,440.2541"
2,VaR Ajustado Estresado,0.0271,"4,824,739.6518"
3,C-VaR,0.0364,"6,478,384.8304"
4,C-VaR Ajustado Promedio,0.0368,"6,555,868.0675"
5,C-VaR Ajustado Estresado,0.0373,"6,646,167.4652"


### Ejercicio 2b

Asume que tienes una sola posición larga en 100 contratos futuros de granos de café, los contratos tienen un nocional de 37,500 unidades cada uno y los precios de cada unidad (libras) de café se encuentran en el excel proporcionado en la hoja DERIVATIVES.

Calcular las métricas de riesgo de mercado y liquidez del portafolio a un día con un 99% de confianza.


In [5]:
data2 = pd.read_excel('DataTarea5.xlsx', sheet_name=1)
data2 = data2.set_index('Date')
posiciones2 = np.array([100*37500])
conf = 0.99
long = True

In [6]:
VaR_APL(data2, posiciones2, conf, long)

,Métrica,Porcentaje,Cash
0,VaR,0.0491,"588,274.1440"
1,VaR Ajustado Promedio,0.0498,"596,455.2298"
2,VaR Ajustado Estresado,0.0511,"612,305.6673"
3,C-VaR,0.0610,"730,740.9163"
4,C-VaR Ajustado Promedio,0.0616,"738,922.0021"
5,C-VaR Ajustado Estresado,0.0630,"754,772.4396"
